## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os 
load_dotenv()

True

In [5]:
# llm = OpenAI(
#     temperature=0,
#     model_name="gpt-3.5-turbo-instruct"
# )


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-002", 
                              google_api_key=os.getenv("GOOGLE_API_KEY"),
                                temperature=0)


In [6]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [8]:
# initialize the math tool

llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

/Users/aelkhodary/Documents/GitHub/Books/DATA/LLM/openai-generative-ai/.venv/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
print(tools[0].name, "---", tools[0].func, "---", tools[0].description)

Calculator --- <bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-002', google_api_key=S

## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [17]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [20]:
#zero_shot_agent("What is root over 25")
zero_shot_agent('What is the average of the numbers 12, 45, and 78?')
#zero_shot_agent("What is the current price of Bitcoin, and how much would it cost if I bought 0.5 BTC today?")



> Entering new AgentExecutor chain...
Thought: I need to sum the numbers and then divide by the count of numbers to find the average.  I will use the calculator for this.
Action: Calculator
Action Input: (12 + 45 + 78) / 3
Observation: Answer: 45.0
Thought:I now know the final answer
Final Answer: 45


> Finished chain.


{'input': 'What is the average of the numbers 12, 45, and 78?', 'output': '45'}

In [13]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
Thought: I need to calculate the area of each room and then sum them up to find the total area of the house.

Action: Calculator

Action Input: (5 * 5) + (5 * 5) + (7 * 6) + (3 * 2)

Observation: Answer: 98
Thought:Thought: I now know the final answer

Final Answer: 98 square meters


> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': '98 square meters'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [23]:
# Define the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

/Users/aelkhodary/Documents/GitHub/Books/DATA/LLM/openai-generative-ai/.venv/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [24]:
# Define the Agent

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [26]:
# Run the Agent with the problem

#agent.run("What is the current price of Bitcoin, and how much would it cost if I bought 0.5 BTC today?")
agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")



> Entering new AgentExecutor chain...
Thought: I need to get the current Microsoft stock price from a reliable source like Google Finance.  Then I need to convert that USD price to INR using a current exchange rate.  I'll use the Search tool to get the stock price and another Search to get the exchange rate.

Action: Search
Action Input: "Microsoft stock price Google Finance"

Observation: Get a real-time Microsoft Corporation (MSFT) stock price quote with breaking news, financials, statistics, charts and more. ... Microsoft and Google beat earnings estimates amid an intensifying rivalry over AI and cloud growth. Google's growth was fueled by AI progress, while Azure remains key for Microsoft. ... integral to daily activities ... While recent third-quarter results appear to show strong AI growth across leaders Alphabet (GOOG-0.02%) (GOOGL 0.10%), Microsoft (MSFT 0.99%), and Meta Platforms (META-0.07%), only Google parent ... Microsoft posted first-quarter earnings of $3.30 per share 

'The Microsoft stock price is approximately $406.35 USD, which is equivalent to approximately ₹34,162.86 INR.'

## Create a custom tool

In [27]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    # model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-002", 
                              google_api_key=os.getenv("GOOGLE_API_KEY"),
                                temperature=0)
    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [29]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [30]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")



> Entering new AgentExecutor chain...
Thought: I need to use the JSON_Response tool to get information about the top football team in the world.  The response should be structured with the three fields specified.

Action: JSON_Response
Action Input: "Which is the top football team in the world and how?"

Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': {'answer': "There is no single definitive answer to the question of which is the top football team in the world.  Different ranking systems exist, using various metrics and weighting factors, and these rankings fluctuate frequently.  Factors considered often include: \n\n* **International Tournament Performance:** Winning major tournaments like the FIFA World Cup or the UEFA Champions League carries significant weight.  However, these tournaments occur infrequently.\n* **League Performance:** Consistent success in top domestic leagues (e.g., Premier Leag

{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': {'answer': "There is no single definitive answer to the question of which is the top football team in the world.  Different ranking systems exist, using various metrics and weighting factors, and these rankings fluctuate frequently.  Factors considered often include: \n\n* **International Tournament Performance:** Winning major tournaments like the FIFA World Cup or the UEFA Champions League carries significant weight.  However, these tournaments occur infrequently.\n* **League Performance:** Consistent success in top domestic leagues (e.g., Premier League, La Liga, Serie A, Bundesliga) is a strong indicator of a team's strength.\n* **Head-to-Head Results:**  Wins against other top teams are considered important, but these are often context-dependent.\n* **Club Elo Ratings:**  These are statistical ratings that attempt to quantify team strength based on past results,